In [45]:
import os
import numpy as np

In [46]:
# Constants
OMEGA_BARYON = 0.046026
C = 300000 # km/s

# file parts here
param_part = '../../param/file_parts/param-mit.part'
doit_part = '../../param/file_parts/doit-mit.part'
hydro_part = '../../param/file_parts/hydro.part'
SIDM_part = '../../param/file_parts/SIDM.part'

# output directory
arepo_out = '../../param/mit/arepo/'
base_out = '../../param/mit/'

In [47]:
# Simulation Parameters
# Change these
N_part = 256
boxsize = 3 # in Mpc

nodes = 2
mem_size = 5 # in GB

DM_type = 'SIDM'
hydro_type = 'HY'
power = 'power0'
sigma=1
extra_info = ''
V_k = 100

heavy_fraction = 0.5

code_directory = 1

In [48]:
#sigmas = [ f'{x:.3}' for x in np.geomspace(0.5, 5, num=10)]
#V_ks = np.geomspace(20, 200, num=10)


directory = f'dir_{code_directory}'

cross_section = f'{power}_sigma{sigma}'
#extra_info = f'_Vkick{V_k:.2f}'

with open(param_part, 'r') as f:
    param_text = f.read()

with open(doit_part, 'r') as f:
    doit_text = f.read()

if DM_type == 'CDM':
    job_string = DM_type + f'_L{boxsize}N{N_part}_' + hydro_type + '_' + directory + extra_info
else:
    job_string = DM_type + f'_L{boxsize}N{N_part}_' + hydro_type + '_' + cross_section + '_' + directory + extra_info
boxsize_kpc = boxsize * 1000
softening_length = boxsize_kpc / (29 * N_part )
mem_size_MB = mem_size * 1000 

# creating output directory
param_name = job_string + '.txt'
out_dir = base_out + f'{job_string}/'

try:
    os.mkdir(out_dir)
except:
    print(f'{out_dir} already exists!')

# replacing text in parameter file
param_text = param_text.replace('{$JOBSTRING}', job_string)
param_text = param_text.replace('{$MEMSIZE}', str(mem_size_MB))
param_text = param_text.replace('{$CODE_DIR}', str(code_directory))
param_text = param_text.replace('{$BOXSIZE}', str(boxsize_kpc))

param_text = param_text.replace('{$SOFTENING_LENGTH}', str(softening_length))
param_text = param_text.replace('{$SOFTENING_LENGTH_HALF}', str(softening_length / 2)) 

if DM_type != 'CDM':
    with open(SIDM_part, 'r') as f:
        param_text += f.read()

if hydro_type == 'HY':
    hydro_flag = 1
    omega_baryon = OMEGA_BARYON
    with open(hydro_part, 'r') as f:
        param_text += f.read()
else:
    hydro_flag = 0
    omega_baryon = 0
param_text = param_text.replace('{$OMEGA_BARYON}', str(omega_baryon))
param_text = param_text.replace('{$HY_FLAG}', str(hydro_flag))

# replacing text in doit script
doit_text = doit_text.replace('{__MEMSIZE__}', f'{mem_size}g')
doit_text = doit_text.replace('{__JOBSTRING__}', job_string)
doit_text = doit_text.replace('{__CODE_DIR__}', str(code_directory))
doit_text = doit_text.replace('{__NODES__}', str(nodes))
doit_text = doit_text.replace('{__PARAM_FILE__}', param_name)
doit_restart_text = doit_text.replace('{__RESTART__}', str(1))
doit_text = doit_text.replace('{__RESTART__}', str(0))

if DM_type == '2cDM':
    delm = 1/2 * (V_k)**2 / C**2

    scatter_states_text = f'0.0 {1 - heavy_fraction}\n{delm:.3} {heavy_fraction}\n'
    scatter_states_out = out_dir + 'scatter_states.txt'
    with open(scatter_states_out, 'w') as f:
        f.write(scatter_states_text)

param_out = out_dir + param_name
param_out1 = arepo_out + param_name
doit_out = out_dir + 'doit.sh'
doit_restart_out = out_dir + 'doit-restart.sh'

with open(param_out, 'w') as f:
    f.write(param_text)
with open(param_out1, 'w') as f:
    f.write(param_text)
with open(doit_out, 'w') as f:
    f.write(doit_text)
with open(doit_restart_out, 'w') as f:
    f.write(doit_restart_text)


../../param/mit/SIDM_L3N256_HY_power0_sigma1_dir_1/ already exists!
